<a href="https://colab.research.google.com/github/fulltrick/-/blob/main/waifast7_gdrive_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
#  Stable Diffusion WebUI (A1111) - Google Drive モデル読み込み版
#  - Googleドライブの特定フォルダにあるモデルを自動検出して使用
#  - ダウンロード不要でセットアップ時間を大幅短縮
#  - Python 3.10 環境で実行
#  - シンボリックリンクでDriveのモデルを直接参照（コピー不要）
# ==========================================================

print("🧪 Notebook kernel Python (参考):")
!python -V

import os, sys, json, shutil, subprocess, time, re, threading
from google.colab import drive, userdata
from IPython.display import display, HTML

# ----------------------------
# 0) Micromamba で Python 3.10 環境を用意
# ----------------------------
print("\n📦 micromamba をセットアップして py310 環境を作成...")
MICROMAMBA = "/usr/local/bin/micromamba"
if not os.path.exists(MICROMAMBA):
    !wget -qO /tmp/micromamba.tar.bz2 https://micromamba.snakepit.net/api/micromamba/linux-64/latest
    !tar -xjf /tmp/micromamba.tar.bz2 -C /usr/local/bin bin/micromamba --strip-components=1

!micromamba create -y -n py310 -c conda-forge python=3.10.13 pip git aria2 curl

print("\n🧪 Runtime (py310) バージョン確認：")
!micromamba run -n py310 python -V

# ----------------------------
# 1) システム側の最低限ツール & Drive
# ----------------------------
print("\n🔧 apt / Google Drive 準備...")
!apt-get update -qq
!apt-get install -y -qq aria2 wget curl

try:
    drive.mount('/content/drive')
    print("✅ Google Drive をマウントしました")
except Exception as e:
    print(f"⚠️ Drive マウント失敗または既にマウント済み: {e}")

# Google Drive のパス設定
# ここを変更することで、異なるフォルダからモデルを読み込めます
drive_base_data_path = '/content/drive/MyDrive/sd_colab_data'
drive_models_path     = os.path.join(drive_base_data_path, 'models/Stable-diffusion')
drive_controlnet_path = os.path.join(drive_base_data_path, 'models/ControlNet')
drive_vae_path        = os.path.join(drive_base_data_path, 'models/VAE')
drive_lora_path       = os.path.join(drive_base_data_path, 'models/Lora')
drive_embeddings_path = os.path.join(drive_base_data_path, 'embeddings')

# フォルダが存在しない場合は作成
os.makedirs(drive_models_path, exist_ok=True)
os.makedirs(drive_controlnet_path, exist_ok=True)
os.makedirs(drive_vae_path, exist_ok=True)
os.makedirs(drive_lora_path, exist_ok=True)
os.makedirs(drive_embeddings_path, exist_ok=True)

print(f"\n📁 Google Drive モデルフォルダ:")
print(f"   Stable Diffusion: {drive_models_path}")
print(f"   ControlNet: {drive_controlnet_path}")
print(f"   VAE: {drive_vae_path}")
print(f"   LoRA: {drive_lora_path}")
print(f"   Embeddings: {drive_embeddings_path}")

# ----------------------------
# 2) Google Drive のモデルを検出
# ----------------------------
def scan_models_in_drive(path, extensions=('.safetensors', '.ckpt', '.pt')):
    """指定されたフォルダ内のモデルファイルをスキャン"""
    if not os.path.exists(path):
        return []
    
    models = []
    for item in os.listdir(path):
        full_path = os.path.join(path, item)
        if os.path.isfile(full_path) and item.endswith(extensions):
            try:
                size_gb = os.path.getsize(full_path) / (1024*1024*1024)
                models.append({
                    'name': item,
                    'path': full_path,
                    'size_gb': size_gb
                })
            except Exception as e:
                print(f"⚠️ {item} の情報取得に失敗: {e}")
    return models

print("\n🔍 Google Drive 内のモデルをスキャン中...")

sd_models = scan_models_in_drive(drive_models_path)
cn_models = scan_models_in_drive(drive_controlnet_path)
vae_models = scan_models_in_drive(drive_vae_path)
lora_models = scan_models_in_drive(drive_lora_path)
embeddings = scan_models_in_drive(drive_embeddings_path, extensions=('.pt', '.safetensors', '.bin'))

print("\n" + "="*70)
print("📊 Google Drive 内のモデル一覧")
print("="*70)

if sd_models:
    print(f"\n🎨 Stable Diffusion モデル ({len(sd_models)}個):")
    for m in sd_models:
        print(f"   ✅ {m['name']} ({m['size_gb']:.2f} GB)")
else:
    print(f"\n⚠️ Stable Diffusion モデルが見つかりません")
    print(f"   → {drive_models_path} にモデルをアップロードしてください")

if cn_models:
    print(f"\n🎮 ControlNet モデル ({len(cn_models)}個):")
    for m in cn_models:
        print(f"   ✅ {m['name']} ({m['size_gb']:.2f} GB)")

if vae_models:
    print(f"\n🖼️ VAE モデル ({len(vae_models)}個):")
    for m in vae_models:
        print(f"   ✅ {m['name']} ({m['size_gb']:.2f} GB)")

if lora_models:
    print(f"\n✨ LoRA モデル ({len(lora_models)}個):")
    for m in lora_models:
        print(f"   ✅ {m['name']} ({m['size_gb']:.2f} GB)")

if embeddings:
    print(f"\n📝 Embeddings ({len(embeddings)}個):")
    for m in embeddings:
        print(f"   ✅ {m['name']} ({m['size_gb']:.2f} GB)")

print("\n" + "="*70)

# モデルが見つからない場合の警告
if not sd_models:
    print("\n⚠️ 重要: Stable Diffusion モデルが見つかりません")
    print("   WebUIは起動できますが、画像生成には最低1つのモデルが必要です。")
    print("   以下のいずれかを実行してください:")
    print("   1. Google Driveにモデルをアップロードして再実行")
    print("   2. WebUI起動後、Civitai Helper拡張から直接ダウンロード")

# ----------------------------
# 3) 既存WebUI削除 → クローン
# ----------------------------
webui_path = '/content/stable-diffusion-webui'
if os.path.exists(webui_path):
    shutil.rmtree(webui_path)
    print(f"\n🧹 既存のWebUIを削除: {webui_path}")

print("\n⬇️ Stable Diffusion WebUI をクローン...")
%cd /content
!git clone -q https://github.com/AUTOMATIC1111/stable-diffusion-webui.git

# ----------------------------
# 4) PyTorch (cu121) を「py310環境」にインストール
# ----------------------------
print("\n🔥 PyTorch (cu121) を py310 環境へインストール...")
!micromamba run -n py310 python -m pip -q install --upgrade pip
!micromamba run -n py310 pip install -q --index-url https://download.pytorch.org/whl/cu121 torch torchvision torchaudio

print("\n📚 追加ライブラリ（py310 環境）をインストール...")
!micromamba run -n py310 pip install -q open_clip_torch transformers timm requests

# ----------------------------
# 5) 拡張（ControlNet / Civitai Helper）
# ----------------------------
print("\n🔌 拡張機能インストール...")
%cd /content/stable-diffusion-webui/extensions
!git clone -q https://github.com/Mikubill/sd-webui-controlnet.git
!git clone -q https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper.git
print("✅ 拡張インストール完了")

# ----------------------------
# 6) Google Drive のモデルをシンボリックリンク
# ----------------------------
print("\n🔗 Google Drive のモデルをWebUIにリンク中...")

def link_models(drive_path, webui_path, models_list, model_type):
    """モデルをシンボリックリンクで接続"""
    if not models_list:
        print(f"   ⏭️ {model_type}: スキップ（モデルなし）")
        return
    
    os.makedirs(webui_path, exist_ok=True)
    linked_count = 0
    
    for model in models_list:
        src = model['path']
        dst = os.path.join(webui_path, model['name'])
        
        try:
            # 既存のリンクやファイルを削除
            if os.path.exists(dst) or os.path.islink(dst):
                os.remove(dst)
            
            # シンボリックリンクを作成
            os.symlink(src, dst)
            linked_count += 1
        except Exception as e:
            print(f"   ⚠️ {model['name']} のリンク失敗: {e}")
    
    print(f"   ✅ {model_type}: {linked_count}個のモデルをリンク")

# 各種モデルをリンク
link_models(
    drive_models_path,
    '/content/stable-diffusion-webui/models/Stable-diffusion',
    sd_models,
    "Stable Diffusion"
)

link_models(
    drive_controlnet_path,
    '/content/stable-diffusion-webui/extensions/sd-webui-controlnet/models',
    cn_models,
    "ControlNet"
)

link_models(
    drive_vae_path,
    '/content/stable-diffusion-webui/models/VAE',
    vae_models,
    "VAE"
)

link_models(
    drive_lora_path,
    '/content/stable-diffusion-webui/models/Lora',
    lora_models,
    "LoRA"
)

link_models(
    drive_embeddings_path,
    '/content/stable-diffusion-webui/embeddings',
    embeddings,
    "Embeddings"
)

print("\n✅ モデルのリンク完了")

# ----------------------------
# 7) WebUI 設定ファイル
# ----------------------------
print("\n⚙️ WebUI 設定ファイルを作成...")
config_content = """{
  "samples_save": true,
  "samples_format": "png",
  "grid_save": true,
  "grid_format": "png",
  "enable_pnginfo": true,
  "save_selected_only": true,
  "jpeg_quality": 80,
  "export_for_4chan": true,
  "img_downscale_threshold": 4.0,
  "target_side_length": 4000,
  "img_max_size_mp": 200,
  "clean_temp_dir_at_start": false
}"""
with open('/content/stable-diffusion-webui/config.json','w') as f:
    f.write(config_content)
print("✅ 設定完了")

# ----------------------------
# 8) インストールサマリー
# ----------------------------
print("\n" + "="*60)
print("🎯 セットアップ完了サマリー:")
print("="*60)
model_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"
if os.path.exists(model_dir):
    model_files = [f for f in os.listdir(model_dir) if f.endswith(('.safetensors','.ckpt'))]
    print("📁 WebUIで利用可能なモデル:")
    if model_files:
        for mf in model_files:
            fp = os.path.join(model_dir, mf)
            is_link = "🔗" if os.path.islink(fp) else "📄"
            try:
                sz = os.path.getsize(fp)/(1024*1024*1024)
                print(f"   {is_link} {mf} ({sz:.2f} GB)")
            except Exception:
                print(f"   {is_link} {mf}")
    else:
        print("   ⚠️ モデルが見つかりません")
else:
    print("   ❌ モデルディレクトリが見つかりません")

ext_dir = "/content/stable-diffusion-webui/extensions"
if os.path.exists(ext_dir):
    extensions = [d for d in os.listdir(ext_dir) if os.path.isdir(os.path.join(ext_dir, d))]
    print("\n🔌 インストールされた拡張機能:")
    for e in extensions: print(f"   ✅ {e}")
print("="*60 + "\n")

# ----------------------------
# 9) WebUI 起動（Python 3.10 で）+ URL検出
# ----------------------------
print("\n🚀 Stable Diffusion WebUI を Python 3.10 で起動します")
%cd /content/stable-diffusion-webui

os.environ['TORCH_COMMAND'] = "pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121"

launch_cmd = [
    "micromamba","run","-n","py310","python","launch.py",
    "--share",
    "--listen",
    "--disable-console-progressbars",
    "--opt-sdp-attention",
    "--enable-insecure-extension-access",
    "--lora-dir","/content/stable-diffusion-webui/models/Lora",
    "--medvram-sdxl",
    "--no-half-vae",
    "--api",
    '--cors-allow-origins=*'
]

print("▶", " ".join(launch_cmd))
print("\n🔍 起動中... Share URLを検出次第表示します\n")

# URLを検出して表示
url_found = {'public': None, 'local': None}

def monitor_output(proc):
    """プロセスの出力を監視してURLを検出"""
    public_pattern = re.compile(r'Running on public URL: (https://[^\s]+)')
    local_pattern = re.compile(r'Running on local URL:\s+(http://[^\s]+)')
    
    while True:
        line = proc.stdout.readline()
        if not line:
            break
        line = line.strip()
        if line:
            print(line)
            
            # Public URLを検出
            match = public_pattern.search(line)
            if match and not url_found['public']:
                url_found['public'] = match.group(1)
                print("\n" + "="*70)
                print("🎉 WebUI が起動しました!")
                print("="*70)
                print(f"\n🌐 Public URL (Share): {url_found['public']}")
                display(HTML(f'<h2 style="color: #4CAF50;">🔗 <a href="{url_found["public"]}" target="_blank">{url_found["public"]}</a></h2>'))
                print("\n" + "="*70 + "\n")
            
            # Local URLを検出
            match = local_pattern.search(line)
            if match and not url_found['local']:
                url_found['local'] = match.group(1)
                if not url_found['public']:  # Public URLがまだ表示されていない場合
                    print(f"\n📍 Local URL: {url_found['local']}")

# プロセスを起動
process = subprocess.Popen(
    launch_cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    universal_newlines=True
)

# 出力監視スレッドを開始
monitor_thread = threading.Thread(target=monitor_output, args=(process,), daemon=True)
monitor_thread.start()

# タイムアウト付きで起動を待つ
timeout = 300  # 5分
start_time = time.time()

while time.time() - start_time < timeout:
    if url_found['public']:
        break
    time.sleep(2)

if not url_found['public']:
    print("\n⚠️ タイムアウト: Share URLが検出されませんでした")
    print("   WebUIは起動中の可能性があります。ログを確認してください。")

print("\n💡 使い方")
print("="*60)
print("📂 モデルの追加方法:")
print(f"   1. Google Driveの以下のフォルダにモデルをアップロード:")
print(f"      {drive_models_path}")
print(f"   2. このノートブックを再実行")
print(f"   3. または、WebUI起動後にCivitai Helper拡張から直接ダウンロード")
print("\n⚡ パフォーマンス:")
print("   - シンボリックリンクを使用しているため、モデルのコピーは不要")
print("   - セットアップ時間が大幅に短縮されます")
print("   - Drive上のモデルを直接読み込むため、ストレージ容量を節約")
print("\n🔧 その他:")
print("   - Python 3.10 環境で実行されています")
print("   - LoRA, VAE, Embeddings も同様にDriveから読み込まれます")
print("="*60)